In [ ]:
import awswrangler as wr
import mlflow

%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

In [ ]:
mlflow_server = "http://localhost:5001"
mlflow.set_tracking_uri(mlflow_server)

In [ ]:
X_train = wr.s3.read_csv("s3://data/final/train/podcast_X_train.csv")
y_train = wr.s3.read_csv("s3://data/final/train/podcast_y_train.csv")

X_test = wr.s3.read_csv("s3://data/final/test/podcast_X_test.csv")
y_test = wr.s3.read_csv("s3://data/final/test/podcast_y_test.csv")

## Investigamos la correlación de features con la variable objetivo

Antes de profundizar en el proceso de construcción de modelo, es esencial comprender las relaciones entre nuestras features  y la variable objetivo. Por lo que vamos a realizar un gráfico que indica el coeficiente de correlación de cada feature en relación con la variable objetivo. Esto nos sirve para:

- Evitar data leakage: Debemos asegurarnos de que ninguna característica se correlacione perfectamente con el objetivo (un coeficiente de correlación de aproximadamente 1.0). Si existe tal correlación, es una señal de que nuestro conjunto de datos podría estar "filtrando" información sobre el objetivo. 

- Garantizar relaciones significativas: Idealmente, nuestras características deberían tener algún grado de correlación con el objetivo. Inclusive si estamos trabajando con un problema de clasificación, aunque los resultados no son tan importantes como en un caso de regresión.

- Auditoría y trazabilidad: Loggear esta visualización de correlación con nuestra ejecución principal de MLflow garantiza la trazabilidad. Proporciona una instantánea de las características de los datos en el momento del entrenamiento del modelo, lo cual es invaluable para propósitos de auditoría y replicabilidad.

In [ ]:
from plots import plot_correlation_with_target, plot_information_gain_with_target

In [ ]:
# Dado que estamos usando como tracking a MLFlow, mostrar los gráficos aquí no tiene sentido.
correlation_plot = plot_correlation_with_target(X_train, y_train)
information_gain_plot = plot_information_gain_with_target(X_train, y_train)

## Arrancamos a experimentar

In [ ]:
import datetime
import optuna

from mlflow.models import infer_signature
from mlflow_aux import get_or_create_experiment

from optuna_aux import champion_callback, objective

from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

Antes de poder realizar experimentos, vamos a crear el experimento en MLFLow, pero para evitar desorden, vamos a usar una función que se fije primero si el experimento existe, si esto es así, devuelve su ID.

Además creamos el nombre del run padre con el que vamos a ir registrando las ejecuciones.

In [ ]:
# Creemos el experimento
experiment_id = get_or_create_experiment("Podcast Data")
print(experiment_id)

run_name_parent = "best_hyperparam_" + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S')

Ya con todo seteado, vamos a ejecutar la optimización usando Optuna, el cual realiza una búsqueda Bayesiana, la cual es más eficiente que una búsqueda de grilla tradicional. La desventaja es que es más difícil de paralelizar.

In [ ]:
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    study = optuna.create_study(direction="maximize")

    study.optimize(lambda trial: objective(trial, X_train, y_train, experiment_id), n_trials=250, callbacks=[champion_callback])

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_train_f1", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Podcast Data",
            "optimizer_engine": "optuna",
            "model_family": "sklearn",
            "feature_set_version": 1,
        }
    )

    # Una vez que terminamos la búsqueda, nos quedamos con el mejor modelo y lo entrenamos
    if study.best_params["classifier"] == "SVC_linear":
        model = SVC(C=study.best_params["svc_c"], kernel='linear', gamma='scale')
    elif study.best_params["classifier"] == "SVC_poly":
        model = SVC(C=study.best_params["svc_c"], kernel='poly',
                    gamma='scale', degree=study.best_params["svc_poly_degree"])
    elif study.best_params["classifier"] == "SVC_rbf":
        model = SVC(C=study.best_params["svc_c"], kernel='rbf', gamma='scale')
    elif study.best_params["classifier"] == "DecisionTreeClassifier":
        model = DecisionTreeClassifier(max_depth=study.best_params["tree_max_depth"])
    else:
        model = RandomForestClassifier(max_depth=study.best_params["rf_max_depth"],
                                       n_estimators=study.best_params["rf_n_estimators"])

    model = model.fit(X_train, y_train.to_numpy().ravel())

    # Y testeamos el modelo y logueamos el resultado
    y_pred = model.predict(X_test)
    f1_score = f1_score(y_test.to_numpy().ravel(), y_pred)
    mlflow.log_metric("test_f1", f1_score)

    # Logueamos los artefactos de las gráficas de correlación y de information_gain
    mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")
    mlflow.log_figure(figure=information_gain_plot, artifact_file="information_gain_plot.png")

    # Guardamos el artefacto del modelo
    artifact_path = "model"

    signature = infer_signature(X_train, model.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="podcast_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)


## Testeando el modelo

Una vez que el modelo fue entrenado, podemos levantarlo y testearlo de una forma agnóstica a donde está guardado.

In [ ]:
loaded = mlflow.sklearn.load_model(model_uri)

In [1]:
import numpy as np
test_data = [0.0, 52.43, 6.35, 0.0, 91.92, 5.99,
             0.0, 15.01, 28.14, 66.05, 48.4, 82.13,
             44.34, 0.0, 37.2, 0.0, 7.63, 74.98,
             34.02, 74.35, 89.41, 0.0, 0.0, 0.0,
             87.92, 0.0, 10.85, 37.55, 18.66, 91.09,
             5.62, 87.52, 0.0, 0.0, 45.55, 96.26, 
             97.0, 0.0, 58.61, 80.24, 0.0, 83.95,
             0.0, 59.53, 78.74, 77.85, 34.65, 0.0,
             89.34, 12.22, 23.33, 60.14, 0.0, 16.37,
             68.62, 43.35, 13.14, 0.0, 26.75, 30.98,
             0.0, 44.59, 70.97, 64.93, 48.84, 0.0, 
             0.0, 99.29, 51.01, 11.43, 50.81, 76.96,
             40.73, 47.57, 53.91, 18.83, 25.61, 0.0,
             14.83, 82.92, 0.0, 43.31, 12.79, 0.0,
             68.25, 0.0, 0.0, 0.0, 34.87, 58.22,
             14.53, 63.64, 17.85, 37.87, 35.89, 75.7,
             0.0, 13.18, 99.79, 0.0, 98.13, 16.58, 
             1.22, 88.88, 0.0, 0.0, 0.0, 16.32,
             29.72, 93.54, 35.85, 46.31, 12.08,
             0.0, 64.16, 71.04, 64.5, 0.0, 7.15,
             2.41, 97.84, 86.23, 0.0, 0.0, 24.56,
             24.6, 41.95, 0.0, 72.34, 95.28, 86.02,
             46.68, 17.86, 50.87, 39.36, 86.69, 39.31,
             0.0, 81.88, 0.0, 0.0, 0.0, 0.0,
             0.0, 0.0, 88.66, 18.31, 43.33, 0.0,
             71.93, 73.78, 4.8, 0.0, 41.27, 22.48,
             79.95, 0.0, 8.27, 94.72, 52.82, 70.26,
             60.05, 0.0, 0.0, 0.0, 30.54, 38.58,
             0.0, 0.0, 82.8, 72.47]
loaded.predict(np.array(test_data).reshape([1, -1]))

NameError: name 'loaded' is not defined

## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()
name = "podcast_model_prod"
desc = "This classifier analyzes podcast data"

# Creamos el modelo productivo
client.create_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = model.get_params()
tags["model"] = type(model).__name__
tags["f1-score"] = f1_score

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "prod", result.version)